# ChannelAttentionFusionT — En İnce Detayına Kadar Yorum

Bu not defteri **yalnızca** `ChannelAttentionFusionT` kodunun açıklamasıdır.

Hedef: Kodun her satırının **ne yaptığını**, hangi tensör şekillerini ürettiğini, ve tasarım tercihinin **neden** böyle olduğunu açık şekilde ortaya koymak.


## 1) Kod (Referans)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


def _softplus_inverse(y: torch.Tensor, eps: float = 1e-6) -> torch.Tensor:
    # softplus(x) = log(1 + exp(x))  ->  inverse: x = log(exp(y) - 1)
    return torch.log(torch.clamp(torch.exp(y) - 1.0, min=eps))


def _get_gate(gate: str):
    g = gate.lower()
    if g == "sigmoid":
        return torch.sigmoid
    if g == "hardsigmoid":
        return F.hardsigmoid
    raise ValueError("gate 'sigmoid' veya 'hardsigmoid' olmalı.")


def _get_act(act: str):
    a = act.lower()
    if a == "relu":
        return nn.ReLU(inplace=True)
    if a == "silu":
        return nn.SiLU(inplace=True)
    raise ValueError("act 'relu' veya 'silu' olmalı.")


class ChannelAttentionFusionT(nn.Module):
    """
    - Sample-wise fusion: avg vs max weights are produced per-sample (B,2) via fusion_router
    - Temperature scaling: optional learnable temperature via inverse-softplus parameterization
    - Debug: if return_fusion_weights=True, forward returns (y, ca, fusion_w) where fusion_w is (B,2)
    """

    def __init__(
        self,
        channels: int,
        reduction: int = 16,
        min_hidden: int = 4,
        fusion: str = "softmax",        # "sum" | "softmax"
        gate: str = "sigmoid",          # "sigmoid" | "hardsigmoid"
        temperature: float = 1.0,
        learnable_temperature: bool = False,
        eps: float = 1e-6,
        act: str = "relu",
        bias: bool = True,
        fusion_router_hidden: int = 16,   # router hidden for sample-wise fusion
        return_fusion_weights: bool = False,
    ):
        super().__init__()

        if fusion not in ("sum", "softmax"):
            raise ValueError("fusion 'sum' veya 'softmax' olmalı.")
        if temperature <= 0:
            raise ValueError("temperature pozitif olmalı.")
        if fusion_router_hidden < 1:
            raise ValueError("fusion_router_hidden >= 1 olmalı.")

        self.eps = float(eps)
        self.fusion = fusion
        self.return_fusion_weights = bool(return_fusion_weights)

        self.gate_fn = _get_gate(gate)

        hidden = max(int(min_hidden), int(channels) // int(reduction))
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)

        self.fc1 = nn.Conv2d(channels, hidden, kernel_size=1, bias=bias)
        self.act = _get_act(act)
        self.fc2 = nn.Conv2d(hidden, channels, kernel_size=1, bias=bias)

        # Sample-wise fusion router (only used if fusion="softmax")
        # Input: cat([avg_s, max_s]) -> (B,2C,1,1)  Output: (B,2,1,1) -> flatten -> (B,2)
        if self.fusion == "softmax":
            self.fusion_router = nn.Sequential(
                nn.Conv2d(2 * channels, fusion_router_hidden, kernel_size=1, bias=True),
                nn.ReLU(inplace=True),
                nn.Conv2d(fusion_router_hidden, 2, kernel_size=1, bias=True),
            )
        else:
            self.fusion_router = None

        # Temperature (optional learnable)
        self.learnable_temperature = bool(learnable_temperature)
        if self.learnable_temperature:
            t0 = torch.tensor(float(temperature))
            t_inv = _softplus_inverse(t0, eps=self.eps)
            self.t_raw = nn.Parameter(t_inv)
        else:
            self.register_buffer("T", torch.tensor(float(temperature)))

    def get_T(self) -> torch.Tensor:
        if self.learnable_temperature:
            return F.softplus(self.t_raw) + self.eps
        return self.T

    def mlp(self, s: torch.Tensor) -> torch.Tensor:
        return self.fc2(self.act(self.fc1(s)))

    def forward(self, x: torch.Tensor):
        # Squeeze
        avg_s = self.avg_pool(x)  # (B,C,1,1)
        max_s = self.max_pool(x)  # (B,C,1,1)

        # Excitation (shared MLP)
        a = self.mlp(avg_s)       # (B,C,1,1)
        m = self.mlp(max_s)       # (B,C,1,1)

        fusion_w = None
        if self.fusion == "sum":
            z = a + m
        else:
            # Sample-wise fusion weights
            s_cat = torch.cat([avg_s, max_s], dim=1)          # (B,2C,1,1) # Bu “tek kanala indirmek” değil; C’yi 2C yapıp daha fazla bilgi vermek.
            logits = self.fusion_router(s_cat).flatten(1)     # (B,2)
            #self.fusion_router(s_cat) çıktısı: şekil olarak (B, 2, 1, 1) gelir (çünkü son conv -> 2)

# Bu logits dediğimiz değerler ağırlık değil.
# Bunlar “ham skor”. Örn:

# logits[b] = [2.3, 0.7] olabilir

# bu “avg daha iyi” gibi bir eğilim taşır ama daha [0,1] aralığında değil, toplamı 1 değil.

            fusion_w = torch.softmax(logits, dim=1)           # (B,2)

# fusion_router softmax değil.

# fusion_router → logit üretir.

# torch.softmax(logits) → ağırlığa çevirir.

# Sonra bu ağırlıklarla a ve m karıştırılır.

            z = fusion_w[:, 0].view(-1, 1, 1, 1) * a + fusion_w[:, 1].view(-1, 1, 1, 1) * m

# # # # # Biz CA bloğunda iki ayrı “kanal önem skoru” üretiyoruz:

# # # # # a = mlp(avg_s) → avg tabanlı kanal logitleri (B, C, 1, 1)

# # # # # m = mlp(max_s) → max tabanlı kanal logitleri (B, C, 1, 1)

# # # # # Bu ikisi aynı şeyi farklı açıdan anlatıyor:

# # # # # avg_s: “genel aktivasyon seviyesi” (dağınık/ortalama bilgi)

# # # # # max_s: “en güçlü aktivasyon” (pik/tepe bilgi)

# # # # # Ama her görüntüde hangisi daha güvenilir sinyal? Aynı değil.

# a: avg yolundan gelen kanal logitleri (B,C,1,1)

# m: max yolundan gelen kanal logitleri (B,C,1,1)

# fusion_w[:,0]: (B,) → sadece avg ağırlığı

# fusion_w[:,1]: (B,) → sadece max ağırlığı

        # Temperature-scaled gating
        T = self.get_T()
        ca = self.gate_fn(z / T)  # (B,C,1,1)
        y = x * ca ## x.shape = (B, C, H, W)  ##  ca.shape = (B, C, 1, 1) 
## Her piksel, kendi kanalının katsayısıyla çarpılıyor. ca → (B, C, 1, 1)  ## otomatik olarak → (B, C, H, W) gibi davranıyor

# # ca = ses açma/kısma düğmesi

# # Kanal 0: ses %90 açık

# # Kanal 1: ses %10 açık

# # Kanal 2: ses %70 açık

# # Kanal 3: neredeyse kapalı

# # y = x * ca tam olarak miksaj masası gibi çalışıyor.

        if self.return_fusion_weights and (fusion_w is not None):
            return y, ca, fusion_w
        return y, ca

## 2) Bu modül ne yapıyor?

Girdi bir özellik haritasıdır:

- `x`: **(B, C, H, W)**

Çıktı iki parçadan oluşur:

- `ca`: kanal maskesi **(B, C, 1, 1)**
- `y`: yeniden ölçeklenmiş çıktı **(B, C, H, W)**

Temel formül:

1. Kanal istatistiklerini çıkar: `avg_s`, `max_s`
2. Bu istatistikleri küçük bir MLP ile kanal logitlerine çevir: `a`, `m`
3. `a` ve `m`’yi birleştir: `z`
4. Temperature ile ölçekle: `z / T`
5. Gate ile [0,1] aralığına taşı: `ca = gate(z / T)`
6. Uygula: `y = x * ca`


## 3) İçe aktarımlar (imports)

```python
import torch
import torch.nn as nn
import torch.nn.functional as F
```

- `torch`: tensör işlemleri
- `nn`: katmanlar ve `nn.Module` altyapısı
- `F`: fonksiyonel API (ör. `softplus`, `hardsigmoid`)


## 4) `_softplus_inverse` neden var?

### Problem: Learnable temperature’ın pozitif kalması gerekir
Temperature (T) bir bölende kullanıldığı için:
- T ≤ 0 olursa matematiksel ve sayısal sorun çıkar.

### Çözüm: `T = softplus(t_raw) + eps`
- `softplus(·)` çıktısı **daima pozitiftir**.
- Böylece `t_raw` serbest (negatif de olabilir), fakat T her zaman > 0.

### Neden inverse?
Başlangıçta T’nin, kullanıcıdan gelen `temperature` değerine eşit olması istenir.  
Bu yüzden `t_raw` şu koşulu sağlayacak şekilde başlatılır:

- `softplus(t_raw) ≈ temperature`

Bunu sağlayan dönüşüm inverse-softplus’tır:

- `t_raw = log(exp(T) - 1)`

### `clamp(..., min=eps)` niye var?
`exp(T) - 1` küçük T değerlerinde sayısal olarak 0’a yaklaşabilir.  
Log içine 0 girmesin diye alt sınır uygulanır.


## 5) `_get_gate` ne yapıyor?

Bu fonksiyon, `gate` parametresine göre attention maskesinin aktivasyonunu seçer:

- `"sigmoid"` → `torch.sigmoid`
- `"hardsigmoid"` → `F.hardsigmoid`

Maskeyi üretirken amaç:
- logitleri [0,1] aralığına taşımak,
- böylece kanal katsayısı gibi kullanabilmek.

Geçersiz string gelirse `ValueError` fırlatır. Bu, hatalı konfigürasyonların eğitim başlamadan yakalanması içindir.


## 6) `_get_act` ne yapıyor?

Bu fonksiyon MLP içindeki aktivasyonu seçer:

- `"relu"` → `nn.ReLU(inplace=True)`
- `"silu"` → `nn.SiLU(inplace=True)`

`inplace=True`:
- bellek kullanımını düşürebilir,
- ancak bazı karma graf senaryolarında debug zorlaştırabilir. Burada tipik kullanım için uygundur.


## 7) Sınıf başlığı ve docstring

`ChannelAttentionFusionT(nn.Module)` bir PyTorch modülüdür.

Docstring’in söylediği üç temel özellik:

1. **Sample-wise fusion**: avg ve max katkıları her örnek için ayrı öğrenilir (B,2).
2. **Temperature scaling**: gating öncesi logit ölçeği T ile kontrol edilir; T opsiyonel öğrenilebilir.
3. **Debug**: istenirse `fusion_w` geri verilir.


## 8) `__init__` parametreleri (ne işe yarar?)

- `channels`: C
- `reduction`: MLP daraltma oranı; hidden yaklaşık `C/reduction`
- `min_hidden`: hidden için alt sınır; çok küçülmeyi engeller
- `fusion`: `"sum"` veya `"softmax"`
- `gate`: sigmoid türü
- `temperature`: başlangıç sıcaklığı
- `learnable_temperature`: T öğrenilsin mi?
- `eps`: sayısal stabilite
- `act`: MLP aktivasyonu
- `bias`: MLP conv’larında bias
- `fusion_router_hidden`: fusion router ara kanal boyutu
- `return_fusion_weights`: debug çıktısı

Bu parametreler hem kapasiteyi (hidden/router_hidden), hem davranışı (fusion/gate), hem de stabiliteyi (temperature/eps) belirler.


## 9) `__init__`: Validasyon blokları

```python
if fusion not in ("sum", "softmax"):
    ...
if temperature <= 0:
    ...
if fusion_router_hidden < 1:
    ...
```

Bu kontrollerin amacı:
- yanlış seçimlerin (örn. temperature=0) eğitim sırasında NaN üretmesine engel olmak,
- konfigürasyonu “fail fast” prensibiyle erken aşamada durdurmak.


## 10) `hidden` hesabı (MLP kapasitesi)

```python
hidden = max(min_hidden, channels // reduction)
```

- `channels // reduction`: SE/CBAM geleneğindeki daraltma
- `min_hidden`: çok küçük kanal sayılarında hidden’ın 0-1 gibi anlamsız değerlere düşmesini engeller

Sonuç:
- Küçük C’de bile MLP tamamen “nefessiz” kalmaz,
- büyük C’de parametre kontrol altında kalır.


## 11) Squeeze katmanları: neden avg ve max?

```python
self.avg_pool = nn.AdaptiveAvgPool2d(1)
self.max_pool = nn.AdaptiveMaxPool2d(1)
```

Squeeze çıktıları:

- `avg_s`: (B,C,1,1) — ortalama aktivasyon
- `max_s`: (B,C,1,1) — en güçlü aktivasyon

İki istatistiğin birlikte kullanılması:
- düz/dağınık aktivasyonlarla (avg) tepe aktivasyonları (max) ayrıştırmaya yardımcı olur.


## 12) MLP (Excitation) katmanları

```python
self.fc1 = nn.Conv2d(C, hidden, 1)
self.act = ...
self.fc2 = nn.Conv2d(hidden, C, 1)
```

Neden `Conv2d(..., kernel_size=1)`?
- Squeeze sonrası tensör 1×1 olduğundan bu yapı efektif olarak “fully-connected” ile aynı işi yapar,
- ama PyTorch’ta kanal ekseninde pratik ve hızlıdır.

Bias:
- `bias` parametresiyle kontrol edilir.
- Bazı tasarımlarda bias maske logitlerinin ofsetini kolaylaştırır.


## 13) Sample-wise fusion router (kritik ek)

Bu router yalnızca `fusion="softmax"` iken kurulur:

```python
Conv2d(2C -> H) + ReLU + Conv2d(H -> 2)
```

**Girdi:** `cat([avg_s, max_s])` = (B,2C,1,1)  
**Çıktı:** (B,2,1,1) → flatten → (B,2)

Bu 2 logit:
- `w_avg`, `w_max` benzeri iki ağırlığın logitidir.
Softmax ile:
- her örnek için `w_avg + w_max = 1` olur.


## 14) Temperature kurulumu: parametre mi buffer mı?

### Learnable ise:
- `self.t_raw = nn.Parameter(...)`
- Gerçek temperature: `softplus(self.t_raw) + eps`

### Sabit ise:
- `self.register_buffer("T", ...)`

**Buffer olması ne demek?**
- Optimizer güncellemez.
- `state_dict` içine girer (model kaydederken T de kaydedilir).


## 15) `get_T()` ayrıntısı

```python
if learnable:
    return softplus(t_raw) + eps
else:
    return T
```

Burada `eps` eklenmesi, T’nin 0’a yaklaşmasını engelleyerek `z/T` oranının aşırı büyümesini azaltır.

Bu, maskenin aşırı saturasyona gitmesini tamamen engellemez; ancak sayısal patlamayı azaltır.


## 16) `mlp(s)` ne bekler?

- `s` şekli: (B,C,1,1)
- dönüş: (B,C,1,1)

Bu fonksiyon aynı MLP’yi hem avg hem max squeeze için kullanır:

- parametre paylaşımı vardır,
- iki squeeze kaynağı aynı “kanal ilişkileri” uzayında işlenir.


## 17) `forward(x)` — adım adım

### 17.1) Squeeze
```python
avg_s = avg_pool(x)  # (B,C,1,1)
max_s = max_pool(x)  # (B,C,1,1)
```

### 17.2) Excitation
```python
a = mlp(avg_s)  # (B,C,1,1)
m = mlp(max_s)  # (B,C,1,1)
```

Bu noktada `a` ve `m` gate öncesi logitlerdir.


## 18) Fusion davranışı

### 18.1) `fusion="sum"`
```python
z = a + m
```
- iki kaynak eşit önemlidir.

### 18.2) `fusion="softmax"`
Router üzerinden örnek-bazlı ağırlık üretilir:

```python
s_cat = cat([avg_s, max_s])      # (B,2C,1,1)
logits = fusion_router(s_cat)    # (B,2,1,1) -> flatten -> (B,2)
fusion_w = softmax(logits, dim=1)# (B,2)
z = w0*a + w1*m
```

`view(-1,1,1,1)`:
- (B,) ağırlığını (B,1,1,1)’e çevirir,
- broadcasting ile (B,C,1,1) ile çarpılabilir hale getirir.


## 19) Temperature + Gate + Uygulama

```python
T = get_T()
ca = gate_fn(z / T)  # (B,C,1,1)
y = x * ca           # (B,C,H,W)
```

- `z/T`: logit ölçeğini ayarlar.
- `gate_fn`: maskeyi [0,1] aralığına taşır.
- `x * ca`: kanal bazında ölçekleme uygular (ca spatial boyutta broadcast edilir).


## 20) Debug çıktısı

```python
if return_fusion_weights and fusion_w is not None:
    return y, ca, fusion_w
return y, ca
```

- `fusion="sum"` ise `fusion_w` yoktur (None).
- `fusion="softmax"` ise `fusion_w` (B,2) döner.

Bu çıktı, router davranışını izlemek içindir:
- hep avg’ye yapışma mı var?
- hep max mi?
- örnekler arasında değişiyor mu?


## 21) İnce Riskler ve Kodun Örtük Varsayımları

1. **Girdi 4D olmalı (B,C,H,W)**  
   Pooling ve Conv2d buna göre çalışır.

2. **Router girişi “squeeze” çıktılarıdır (avg_s, max_s)**  
   Router, MLP sonrası `a,m` yerine squeeze’i kullanır. Bu, karar mekanizmasının “ham istatistiklere” dayanması demektir.

3. **Saturasyon riski**  
   `sigmoid` büyük |z/T| değerlerinde 0/1’e yapışır. Temperature bunu yumuşatabilir; ancak tek başına garantili çözüm değildir.

4. **Örnek-bazlı ağırlıklar**  
   `fusion_w` softmax olduğu için her örnekte iki ağırlığın toplamı 1’dir. Bu, ölçek patlamasını sınırlayan bir normalizasyondur.


## 22) Çıkış şekilleri (hızlı kontrol)

- `x`: (B,C,H,W)
- `avg_s`, `max_s`: (B,C,1,1)
- `a`, `m`: (B,C,1,1)
- `fusion_w` (softmax modunda): (B,2)
- `z`: (B,C,1,1)
- `ca`: (B,C,1,1)
- `y`: (B,C,H,W)
